In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [3]:
!unzip "/content/drive/My Drive/YOLO/rock-paper-scissors_1.zip" -d "/content/original_data"
!unzip "/content/drive/My Drive/YOLO/RSP/4class_data.zip" -d "/content/new_data"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: /content/original_data/train/labels/restaurant_04_15_altavista_jpg.rf.0077821b58455bac9395ff4c37ed59f6.txt  
 extracting: /content/original_data/train/labels/restaurant_04_15_altavista_jpg.rf.4a5adc90a49c0d185f57df5ecd103394.txt  
 extracting: /content/original_data/train/labels/restaurant_04_15_altavista_jpg.rf.4cd26d5a3d208c8a5da8e9f3c1413ce0.txt  
 extracting: /content/original_data/train/labels/restaurant_04_15_altavista_jpg.rf.792aa91f2f1cc6528dd60955627ab70f.txt  
 extracting: /content/original_data/train/labels/restaurant_04_15_altavista_jpg.rf.83aed99b940cfe0fb4908f9307438f37.txt  
 extracting: /content/original_data/train/labels/restaurant_04_15_altavista_jpg.rf.eca1076fbfc915cb470965750da690b2.txt  
 extracting: /content/original_data/train/labels/restaurant_04_15_altavista_jpg.rf.ed1a52ff92a67ff59eda2ddb5a4aa0ca.txt  
 extracting: /content/original_data/train/labels/restaurant_07_18_altavista_jpg.rf.18a540e10fdc3cad4163b2a10b4

In [4]:
!pip install albumentations -q

In [5]:
# 데이터 증강 및 데이터셋 새롭게 구성
import os
import glob
import shutil
import numpy as np
from tqdm import tqdm
import cv2
import albumentations as A

# 경로 설정
original_dataset_path = '/content/original_data'
new_dataset_path = '/content/new_data'
rsp_data_path = '/content/rsp_data'

# rsp_data 내 train, valid, test 폴더 및 images, labels 폴더 생성
def create_dataset_dirs(base_path):
    for split in ['train', 'valid', 'test']:
        os.makedirs(os.path.join(base_path, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(base_path, split, 'labels'), exist_ok=True)

create_dataset_dirs(rsp_data_path)

# 1. 새로운 4클래스 데이터 증강 및 분배
print("1. 새로운 4클래스 데이터 처리 중...")

# 증강 변환 정의
augmentations = [
    A.Compose([A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1)]),
    A.Compose([A.GaussianBlur(blur_limit=(3, 7), p=1)]),
    A.Compose([A.HueSaturationValue(hue_shift_limit=0.03, sat_shift_limit=0.6, val_shift_limit=0.5, p=1)]),
    A.Compose([A.Rotate(limit=20, p=1)]),
    A.Compose([A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=1)])
]

# 이미지 및 라벨 파일 목록
img_files = sorted(glob.glob(os.path.join(new_dataset_path, 'img', '*.jpg')))
print(f"새로운 4클래스 이미지 수: {len(img_files)}")

# 6:2:2 비율로 분할
np.random.seed(42)
np.random.shuffle(img_files)

train_split = int(len(img_files) * 0.6)
valid_split = int(len(img_files) * 0.8)

train_files = img_files[:train_split]
valid_files = img_files[train_split:valid_split]
test_files = img_files[valid_split:]

# 증강 및 저장 함수 정의
def augment_and_save(file_list, split):
    for img_path in tqdm(file_list, desc=f'Augmenting {split}'):
        img = cv2.imread(img_path)
        if img is None:
            continue

        base_name = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(new_dataset_path, 'label', f'{base_name}.txt')

        if not os.path.exists(label_path):
            continue

        with open(label_path, 'r') as f:
            label_content = f.read().strip()

        # 원본 복사
        dst_img_path = os.path.join(rsp_data_path, split, 'images', os.path.basename(img_path))
        dst_label_path = os.path.join(rsp_data_path, split, 'labels', f'{base_name}.txt')
        shutil.copy(img_path, dst_img_path)
        with open(dst_label_path, 'w') as f:
            f.write(label_content)

        # 증강 적용
        for i, aug in enumerate(augmentations):
            augmented = aug(image=img)
            aug_img = augmented['image']
            aug_img_name = f'{base_name}_aug{i+1}.jpg'
            aug_label_name = f'{base_name}_aug{i+1}.txt'
            cv2.imwrite(os.path.join(rsp_data_path, split, 'images', aug_img_name), aug_img)
            with open(os.path.join(rsp_data_path, split, 'labels', aug_label_name), 'w') as f:
                f.write(label_content)

# 각 분할에 대해 증강 적용
augment_and_save(train_files, 'train')
augment_and_save(valid_files, 'valid')
augment_and_save(test_files, 'test')

# 2. 기존 데이터셋 재분배 (6:2:2 비율)
print("\n2. 기존 데이터셋 재분배 중...")

# 기존 데이터셋의 모든 이미지와 라벨 파일 수집
all_original_images = []
all_original_labels = []

for split in ['train', 'valid', 'test']:
    img_dir = os.path.join(original_dataset_path, split, 'images')
    label_dir = os.path.join(original_dataset_path, split, 'labels')

    if os.path.exists(img_dir) and os.path.exists(label_dir):
        img_files = glob.glob(os.path.join(img_dir, '*.jpg'))

        for img_path in img_files:
            base_name = os.path.splitext(os.path.basename(img_path))[0]
            label_path = os.path.join(label_dir, f'{base_name}.txt')

            if os.path.exists(label_path):
                all_original_images.append(img_path)
                all_original_labels.append(label_path)

print(f"기존 데이터셋 이미지 수: {len(all_original_images)}")

# 6:2:2 비율로 재분할
if len(all_original_images) > 0:
    combined = list(zip(all_original_images, all_original_labels))
    np.random.seed(42)
    np.random.shuffle(combined)
    all_original_images, all_original_labels = zip(*combined)

    # 분할 인덱스 계산
    train_idx = int(len(all_original_images) * 0.6)
    valid_idx = int(len(all_original_images) * 0.8)

    # 분할
    train_imgs = all_original_images[:train_idx]
    train_labels = all_original_labels[:train_idx]

    valid_imgs = all_original_images[train_idx:valid_idx]
    valid_labels = all_original_labels[train_idx:valid_idx]

    test_imgs = all_original_images[valid_idx:]
    test_labels = all_original_labels[valid_idx:]

    # 복사 함수
    def copy_files_to_split(imgs, labels, split):
        for img_path, label_path in tqdm(zip(imgs, labels), desc=f'Copying to {split}', total=len(imgs)):
            img_name = os.path.basename(img_path)
            label_name = os.path.basename(label_path)

            dst_img_path = os.path.join(rsp_data_path, split, 'images', img_name)
            dst_label_path = os.path.join(rsp_data_path, split, 'labels', label_name)

            shutil.copy(img_path, dst_img_path)
            shutil.copy(label_path, dst_label_path)

    # 각 분할에 파일 복사
    copy_files_to_split(train_imgs, train_labels, 'train')
    copy_files_to_split(valid_imgs, valid_labels, 'valid')
    copy_files_to_split(test_imgs, test_labels, 'test')
else:
    print("기존 데이터셋이 비어있습니다. 새로운 4클래스 데이터만 사용합니다.")

# 3. data.yaml 파일 생성
yaml_content = f"""train: {os.path.join(rsp_data_path, 'train', 'images')}
val: {os.path.join(rsp_data_path, 'valid', 'images')}
test: {os.path.join(rsp_data_path, 'test', 'images')}

nc: 4
names: ['paper', 'rock', 'scissors', 'justhand']
"""

with open(os.path.join(rsp_data_path, 'data.yaml'), 'w') as f:
    f.write(yaml_content)

# 결과 출력
train_count = len(glob.glob(os.path.join(rsp_data_path, 'train', 'images', '*.jpg')))
valid_count = len(glob.glob(os.path.join(rsp_data_path, 'valid', 'images', '*.jpg')))
test_count = len(glob.glob(os.path.join(rsp_data_path, 'test', 'images', '*.jpg')))

print("\n최종 데이터셋 통계:")
print(f"훈련 이미지: {train_count}")
print(f"검증 이미지: {valid_count}")
print(f"테스트 이미지: {test_count}")
print(f"총 이미지: {train_count + valid_count + test_count}")
print("\nYOLO v11 학습을 위한 데이터셋이 성공적으로 준비되었습니다.")





/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


1. 새로운 4클래스 데이터 처리 중...
새로운 4클래스 이미지 수: 383


Augmenting test: 100%|██████████| 77/77 [00:01<00:00, 64.99it/s]



2. 기존 데이터셋 재분배 중...
기존 데이터셋 이미지 수: 15874


Copying to test: 100%|██████████| 3175/3175 [00:00<00:00, 4705.30it/s]



최종 데이터셋 통계:
훈련 이미지: 10898
검증 이미지: 3637
테스트 이미지: 3637
총 이미지: 18172

YOLO v11 학습을 위한 데이터셋이 성공적으로 준비되었습니다.


In [6]:
# YOLO11n 모델 학습 코드 - Google Colab T4 환경 최적화
from ultralytics import YOLO
import os
from datetime import datetime

# 필요한 패키지 설치
!pip install ultralytics -q

# 데이터셋 경로 설정
data_yaml_path = '/content/rsp_data/data.yaml'

# 현재 시간으로 실험 이름 생성
exp_name = f"rsp_yolo11n_{datetime.now().strftime('%Y%m%d_%H%M')}"

# 사전 훈련된 YOLO11n 모델 로드
model = YOLO('yolo11n.pt')

# 학습 실행
results = model.train(
    data=data_yaml_path,
    epochs=100,                # 에폭 수
    patience=15,               # 조기 종료 인내심
    batch=16,                  # 배치 크기 (T4 메모리에 최적화)
    imgsz=640,                 # 입력 이미지 크기
    save=True,                 # 모델 저장
    save_period=10,            # 10 에폭마다 모델 저장
    cache=True,                # 캐싱으로 학습 속도 향상
    device=0,                  # GPU 사용
    workers=8,                 # 데이터 로딩 워커 수
    pretrained=True,           # 사전 훈련된 가중치 사용
    optimizer='Adam',          # 최적화 알고리즘
    lr0=0.001,                 # 초기 학습률
    lrf=0.01,                  # 최종 학습률 (초기 학습률의 비율)
    momentum=0.937,            # SGD 모멘텀
    weight_decay=0.0005,       # 가중치 감쇠
    warmup_epochs=3.0,         # 웜업 에폭
    warmup_momentum=0.8,       # 웜업 모멘텀
    warmup_bias_lr=0.1,        # 웜업 바이어스 학습률
    box=7.5,                   # 박스 손실 가중치
    cls=0.5,                   # 클래스 손실 가중치
    dfl=1.5,                   # DFL 손실 가중치
    label_smoothing=0.0,       # 라벨 스무딩
    nbs=64,                    # 공칭 배치 크기
    hsv_h=0.015,               # HSV 색조 증강
    hsv_s=0.7,                 # HSV 채도 증강
    hsv_v=0.4,                 # HSV 값 증강
    translate=0.1,             # 이미지 이동 증강
    scale=0.5,                 # 이미지 스케일 증강
    fliplr=0.5,                # 좌우 반전 확률
    mosaic=1.0,                # 모자이크 증강
    mixup=0.1,                 # 믹스업 증강
    copy_paste=0.1,            # 복사-붙여넣기 증강
    auto_augment='randaugment', # 자동 증강 정책
    erasing=0.4,               # 랜덤 지우기 확률
    close_mosaic=10,           # 마지막 에폭에서 모자이크 비활성화
    project='rsp_detection',   # 프로젝트 이름
    name=exp_name,             # 실험 이름
    fraction=1.0               # 전체 데이터셋 사용
)

# 모델 평가
metrics = model.val()
print(f"mAP50-95: {metrics.box.map}")
print(f"mAP50: {metrics.box.map50}")
print(f"Precision: {metrics.box.p}")
print(f"Recall: {metrics.box.r}")

# 모델 내보내기 (ONNX 형식)
model.export(format='onnx', dynamic=True, simplify=True)

# 학습된 모델 경로 출력
print(f"\n학습된 모델 경로:")
print(f"Best 모델: {model.best}")
print(f"ONNX 모델: {os.path.splitext(model.best)[0]}.onnx")

# Google Drive에 모델 저장 (선택사항)
from google.colab import drive
drive.mount('/content/drive')
save_path = '/content/drive/MyDrive/yolo11n_models'
os.makedirs(save_path, exist_ok=True)
!cp {model.best} {save_path}/
!cp {os.path.splitext(model.best)[0]}.onnx {save_path}/
print(f"모델이 {save_path}에 저장되었습니다.")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 147MB/s]


WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/rsp_data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=rsp_yolo11n_20250515_0515, nbs=64, nms=False, opset=None, opti

100%|██████████| 755k/755k [00:00<00:00, 31.2MB/s]

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

...

val: Scanning /content/rsp_data/valid/labels.cache... 3637 images, 1282 backgrounds, 0 corrupt: 100%|██████████| 3637/3637 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (4.0GB RAM): 100%|██████████| 3637/3637 [00:01<00:00, 2215.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 228/228 [00:13<00:00, 17.39it/s]


                   all       3637       2657      0.981      0.986      0.994      0.842
                 paper        675        692      0.979      0.986      0.994      0.855
                  rock        928       1067      0.982      0.981      0.993      0.845
              scissors        715        748      0.983      0.985      0.993      0.864
              justhand        150        150       0.98      0.992      0.994      0.803
Speed: 0.2ms preprocess, 0.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to rsp_detection/rsp_yolo11n_20250515_05152
mAP50-95: 0.8416470821066759
mAP50: 0.9937975032355181
Precision: [    0.97921     0.98201     0.98281     0.98025]
Recall: [    0.98555     0.98126     0.98529     0.99245]
Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel CPUs. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from

AttributeError: 'DetectionModel' object has no attribute 'best'

In [7]:
!zip -r /content/rsp_detection.zip /content/rsp_detection

  adding: content/rsp_detection/ (stored 0%)
  adding: content/rsp_detection/rsp_yolo11n_20250515_0515/ (stored 0%)
  adding: content/rsp_detection/rsp_yolo11n_20250515_0515/confusion_matrix_normalized.png (deflated 28%)
  adding: content/rsp_detection/rsp_yolo11n_20250515_0515/train_batch0.jpg (deflated 3%)
  adding: content/rsp_detection/rsp_yolo11n_20250515_0515/val_batch2_pred.jpg (deflated 10%)
  adding: content/rsp_detection/rsp_yolo11n_20250515_0515/labels.jpg (deflated 26%)
  adding: content/rsp_detection/rsp_yolo11n_20250515_0515/args.yaml (deflated 52%)
  adding: content/rsp_detection/rsp_yolo11n_20250515_0515/val_batch2_labels.jpg (deflated 10%)
  adding: content/rsp_detection/rsp_yolo11n_20250515_0515/P_curve.png (deflated 17%)
  adding: content/rsp_detection/rsp_yolo11n_20250515_0515/F1_curve.png (deflated 11%)
  adding: content/rsp_detection/rsp_yolo11n_20250515_0515/train_batch1.jpg (deflated 4%)
  adding: content/rsp_detection/rsp_yolo11n_20250515_0515/val_batch1_labels

In [ ]:
# best2.pt
# 데이터셋 가져와서 yaml 파일 확인

import os
from ultralytics import YOLO

# 현재 작업 디렉토리 확인
current_dir = os.getcwd()
print(f"현재 작업 디렉토리: {current_dir}")

# 데이터셋 경로 설정
dataset_path = os.path.join(current_dir, "")  # 데이터셋 루트 경로
yaml_path = os.path.join(dataset_path, "data.yaml")

# YAML 파일 존재 여부 확인
if os.path.exists(yaml_path):
    print(f"YAML 파일 경로: {yaml_path}")
else:
    print(f"YAML 파일을 찾을 수 없습니다: {yaml_path}")

# 메모리 관리 설정
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 모델 초기화
model = YOLO('yolo11n.pt')  # 더 작은 모델 사용

# 학습 실행
results = model.train(
    data=yaml_path,
    epochs=50,       # 에포크 수 감소
    imgsz=480,       # 이미지 크기 감소
    batch=-1,        # 자동 배치 크기 조정
    patience=15,     # 조기 종료
    device=0,        # 단일 GPU 사용
    save=True,
    project='rps_models',
    name='model_t4_optimized'
)



현재 작업 디렉토리: /content
YAML 파일 경로: /content/data.yaml
New https://pypi.org/project/ultralytics/8.3.134 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=480, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=model_t4_opt

train: Scanning /content/train/labels.cache... 14966 images, 5957 backgrounds, 0 corrupt: 100%|██████████| 14966/14966 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=480 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (Tesla T4) 14.74G total, 0.10G reserved, 0.05G allocated, 14.59G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     2590425       3.624         0.373         41.44           218        (1, 3, 480, 480)                    list
     2590425       7.248         0.455         31.23         114.2        (2, 3, 480, 480)                    list
     2590425        14.5         0.617         47.37         161.7        (4, 3, 480, 480)                    list
     2590425       28.99         0.927         40.33         164.4        (8, 3, 480, 480)                    list
     2590425       57.98         1.

train: Scanning /content/train/labels.cache... 14966 images, 5957 backgrounds, 0 corrupt: 100%|██████████| 14966/14966 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

      50/50      7.73G     0.5336     0.2547     0.9071          4        480: 100%|██████████| 137/137 [02:43<00:00,  1.19s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:05<00:00,  1.91s/it]

                   all        588        406      0.955      0.928      0.956      0.764



50 epochs completed in 2.417 hours.
Optimizer stripped from rps_models/model_t4_optimized2/weights/last.pt, 5.4MB
Optimizer stripped from rps_models/model_t4_optimized2/weights/best.pt, 5.4MB

Validating rps_models/model_t4_optimized2/weights/best.pt...
Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:07<00:00,  2.59s/it]


                   all        588        406      0.957      0.924      0.955      0.767
                 paper        134        141       0.97      0.925       0.96      0.765
                  rock        125        147       0.95      0.932      0.952      0.755
              scissors        114        118      0.951      0.915      0.954      0.782
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to rps_models/model_t4_optimized2


In [4]:
!unzip "/content/drive/My Drive/YOLO/RSP/collected_data.zip" -d "/content/"

Archive:  /content/drive/My Drive/YOLO/RSP/collected_data.zip
   creating: /content/train/
   creating: /content/train/images/
  inflating: /content/train/images/img_00000.jpg  
  inflating: /content/train/images/img_00001.jpg  
  inflating: /content/train/images/img_00002.jpg  
  inflating: /content/train/images/img_00003.jpg  
  inflating: /content/train/images/img_00004.jpg  
  inflating: /content/train/images/img_00005.jpg  
  inflating: /content/train/images/img_00006.jpg  
  inflating: /content/train/images/img_00007.jpg  
  inflating: /content/train/images/img_00008.jpg  
  inflating: /content/train/images/img_00009.jpg  
  inflating: /content/train/images/img_00019.jpg  
  inflating: /content/train/images/img_00020.jpg  
  inflating: /content/train/images/img_00021.jpg  
  inflating: /content/train/images/img_00023.jpg  
  inflating: /content/train/images/img_00024.jpg  
  inflating: /content/train/images/img_00025.jpg  
  inflating: /content/train/images/img_00026.jpg  
  infl

In [5]:
# best3.pt
# best2.pt 가져와서 그...collected data 학습
import os
from ultralytics import YOLO

# 현재 작업 디렉토리 확인
current_dir = os.getcwd()
print(f"현재 작업 디렉토리: {current_dir}")

# 데이터셋 경로 설정
dataset_path = os.path.join(current_dir, "")  # 데이터셋 루트 경로
yaml_path = os.path.join(dataset_path, "data.yaml")

# YAML 파일 존재 여부 확인
if os.path.exists(yaml_path):
    print(f"YAML 파일 경로: {yaml_path}")
else:
    print(f"YAML 파일을 찾을 수 없습니다: {yaml_path}")

# 메모리 관리 설정
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 모델 초기화
model = YOLO('best2.pt')  # 더 작은 모델 사용

# 학습 실행
results = model.train(
    data='data.yaml',
    epochs=30,        # 너무 많은 에폭은 과적합 위험
    imgsz=480,
    batch=4,          # 이미지 수가 적다면 작은 배치 추천
    patience=10,      # early stopping
    device=0,
    lr0=1e-4,         # 학습률 낮추기 → 기존 가중치 보존하면서 조금씩 학습
    save=True,
    project='rps_models',
    name='finetuned_small_56'
)




현재 작업 디렉토리: /content
YAML 파일 경로: /content/data.yaml
Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=480, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=best2.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=finetuned_small_562, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspect

100%|██████████| 755k/755k [00:00<00:00, 85.1MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

YOLO11n summary: 181 layers, 2,590,425 parameters, 2,590,409 gradients, 6.4 GFLOPs

Transferred 499/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 354MB/s]


...

      24/30     0.768G      0.409     0.2491     0.8775          3        480: 100%|██████████| 12/12 [00:01<00:00,  6.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 18.22it/s]

                   all          9          9      0.986          1      0.995      0.887
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 14, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



24 epochs completed in 0.014 hours.
Optimizer stripped from rps_models/finetuned_small_562/weights/last.pt, 5.4MB
Optimizer stripped from rps_models/finetuned_small_562/weights/best.pt, 5.4MB

Validating rps_models/finetuned_small_562/weights/best.pt...
Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 14.96it/s]


                   all          9          9      0.984          1      0.995      0.913
                  rock          4          4      0.979          1      0.995      0.862
              scissors          5          5       0.99          1      0.995      0.964
Speed: 0.1ms preprocess, 5.1ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to rps_models/finetuned_small_562


In [ ]:
# best1.pt
!pip install ultralytics
import os
from ultralytics import YOLO

# Roboflow에서 데이터셋 다운로드
!pip install roboflow
from roboflow import Roboflow

# API 키로 Roboflow 초기화
rf = Roboflow(api_key="p2C1Qr5lCgNbiEcup6JK")

# 먼저 사용 가능한 프로젝트 확인
workspace = rf.workspace("yolo-2hpdp")
projects = workspace.projects()
print("사용 가능한 프로젝트:", projects)

# 프로젝트 선택
project = rf.workspace("yolo-2hpdp").project("rock-paper-scissors-sxsw-kljj7")

# 사용 가능한 버전 확인
versions = project.versions()
print("사용 가능한 버전:", [v.version for v in versions])

# 존재하는 버전 번호 사용 (예: 가장 최신 버전)
latest_version = versions[-1].version
print(f"최신 버전: {latest_version}")

# 존재하는 버전으로 다운로드
dataset = project.version(latest_version).download("yolov8")

# 다운로드된 데이터셋 경로 확인
print(f"데이터셋 위치: {dataset.location}")
!ls -la {dataset.location}

# 실제 디렉토리 구조 확인
print("디렉토리 구조:")
!find {dataset.location} -type d | sort

# YAML 파일 내용 확인
yaml_path = f"{dataset.location}/data.yaml"
print("원본 YAML 내용:")
!cat {yaml_path}

# YAML 파일 수정
import yaml

# YAML 파일 읽기
with open(yaml_path, 'r') as file:
    data = yaml.safe_load(file)

# 경로 수정 - 실제 존재하는 디렉토리 사용
train_path = f"{dataset.location}/train/images"
test_path = f"{dataset.location}/test/images"

# valid 디렉토리가 있는지 확인
valid_path = f"{dataset.location}/valid/images"
if os.path.exists(valid_path):
    val_path = valid_path
else:
    # valid 디렉토리가 없으면 test 디렉토리를 검증 데이터셋으로 사용
    val_path = test_path
    print("'valid' 디렉토리가 없어 'test' 디렉토리를 검증 데이터셋으로 사용합니다.")

# 절대 경로로 업데이트
data['train'] = train_path
data['val'] = val_path
data['test'] = test_path

# 수정된 YAML 저장
with open(yaml_path, 'w') as file:
    yaml.dump(data, file)

print("\n수정된 YAML 내용:")
!cat {yaml_path}

# 경로 확인
print(f"\n훈련 데이터 경로 존재 여부: {os.path.exists(train_path)}")
print(f"검증 데이터 경로 존재 여부: {os.path.exists(val_path)}")
print(f"테스트 데이터 경로 존재 여부: {os.path.exists(test_path)}")

# 모델 학습
model = YOLO('yolo11n.pt')  # 사전 학습된 모델 로드
results = model.train(
    data=yaml_path,
    epochs=100,
    imgsz=640,
    batch=16,
    patience=20,
    save=True,
    project='rps_models',
    name='model1'
)
# 학습된 모델 평가
model.val()

# 최종 모델 파일 경로 확인
best_model_path = f"rps_models/model1/weights/best.pt"
print(f"학습된 모델 경로: {best_model_path}")

# 구글 드라이브에 모델 저장 (선택사항)
from google.colab import drive
drive.mount('/content/drive')
!cp {best_model_path} /content/drive/MyDrive/



loading Roboflow workspace...
사용 가능한 프로젝트: ['yolo-2hpdp/rock-paper-scissors-sxsw-kljj7', 'yolo-2hpdp/temp-mi6mx', 'yolo-2hpdp/vehicles-q0x2v-n6vpx']
loading Roboflow workspace...
loading Roboflow project...
사용 가능한 버전: ['2', '1']
최신 버전: 1
데이터셋 위치: /content/rock-paper-scissors-1
total 250476
drwxr-xr-x 4 root root      4096 May 13 22:03 .
drwxr-xr-x 1 root root      4096 May 13 22:04 ..
-rw-r--r-- 1 root root       383 May 13 22:08 data.yaml
-rw-r--r-- 1 root root      1854 May 13 22:03 README.dataset.txt
-rw-r--r-- 1 root root      1206 May 13 22:03 README.roboflow.txt
-rw-r--r-- 1 root root 256458719 May 13 22:03 roboflow.zip
drwxr-xr-x 4 root root      4096 May 13 22:03 test
drwxr-xr-x 4 root root      4096 May 13 22:03 train
디렉토리 구조:
/content/rock-paper-scissors-1
/content/rock-paper-scissors-1/test
/content/rock-paper-scissors-1/test/images
/content/rock-paper-scissors-1/test/labels
/content/rock-paper-scissors-1/train
/content/rock-paper-scissors-1/train/images
/content/rock-paper-

100%|██████████| 755k/755k [00:00<00:00, 19.9MB/s]

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

YOLO11n summary: 181 layers, 2,590,425 parameters, 2,590,409 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1427.5±515.6 MB/s, size: 49.2 KB)


train: Scanning /content/rock-paper-scissors-1/train/labels... 2571 images, 4884 backgrounds, 0 corrupt: 100%|██████████| 6517/6517 [00:01<00:00, 3580.32it/s]


...


100 epochs completed in 3.255 hours.
Optimizer stripped from rps_models/model14/weights/last.pt, 5.5MB
Optimizer stripped from rps_models/model14/weights/best.pt, 5.5MB

Validating rps_models/model14/weights/best.pt...
Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:05<00:00,  2.13it/s]


                   all        329        217      0.916      0.357      0.472      0.421
                 Paper         73         73      0.951      0.315      0.435      0.391
                  Rock         66         75      0.876      0.307      0.418      0.361
              Scissors         67         69      0.921      0.449      0.564      0.512
Speed: 0.3ms preprocess, 3.9ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to rps_models/model14
Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 650.3±169.8 MB/s, size: 31.6 KB)


val: Scanning /content/rock-paper-scissors-1/test/labels.cache... 329 images, 134 backgrounds, 0 corrupt: 100%|██████████| 329/329 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:06<00:00,  3.45it/s]


                   all        329        217      0.916      0.357      0.472      0.422
                 Paper         73         73      0.951      0.315      0.434      0.392
                  Rock         66         75      0.876      0.307      0.418      0.361
              Scissors         67         69      0.921      0.449      0.564      0.512
Speed: 3.1ms preprocess, 6.0ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to rps_models/model142
학습된 모델 경로: rps_models/model1/weights/best.pt
Mounted at /content/drive
cp: cannot stat 'rps_models/model1/weights/best.pt': No such file or directory


In [ ]:
!zip -r rps_models.zip rps_models/

  adding: rps_models/ (stored 0%)
  adding: rps_models/model13/ (stored 0%)
  adding: rps_models/model13/weights/ (stored 0%)
  adding: rps_models/model13/args.yaml (deflated 52%)
  adding: rps_models/model14/ (stored 0%)
  adding: rps_models/model14/val_batch2_labels.jpg (deflated 9%)
  adding: rps_models/model14/val_batch2_pred.jpg (deflated 9%)
  adding: rps_models/model14/confusion_matrix.png (deflated 33%)
  adding: rps_models/model14/val_batch1_labels.jpg (deflated 8%)
  adding: rps_models/model14/train_batch2.jpg (deflated 2%)
  adding: rps_models/model14/train_batch1.jpg (deflated 5%)
  adding: rps_models/model14/val_batch0_pred.jpg (deflated 12%)
  adding: rps_models/model14/train_batch36720.jpg (deflated 9%)
  adding: rps_models/model14/train_batch36722.jpg (deflated 6%)
  adding: rps_models/model14/train_batch0.jpg (deflated 3%)
  adding: rps_models/model14/results.csv (deflated 59%)
  adding: rps_models/model14/R_curve.png (deflated 19%)
  adding: rps_models/model14/F1_curv

In [ ]:
# 모델 1: Laurence Moroney의 Rock Paper Scissors 데이터셋

# 코랩에서 실행
!git clone https://github.com/lmoroney/dlaicourse.git
!pip install ultralytics

# 데이터셋 준비 (YOLO 형식으로 변환)
import os
import shutil
import random
from PIL import Image
import numpy as np

# 데이터셋 경로
dataset_path = "dlaicourse/python/rockpaperscissors"
output_path = "rps_dataset_yolo"

# YOLO 형식 디렉토리 구조 생성
os.makedirs(f"{output_path}/images/train", exist_ok=True)
os.makedirs(f"{output_path}/images/val", exist_ok=True)
os.makedirs(f"{output_path}/labels/train", exist_ok=True)
os.makedirs(f"{output_path}/labels/val", exist_ok=True)

# 클래스 매핑
class_mapping = {
    'rock': 0,
    'paper': 1,
    'scissors': 2
}

# 검증 세트 비율
val_ratio = 0.2

# 데이터셋 변환 함수
def convert_to_yolo_format():
    file_count = 0

    # 각 클래스 폴더 처리
    for class_name in class_mapping.keys():
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.exists(class_path):
            print(f"경로를 찾을 수 없음: {class_path}")
            continue

        # 클래스 폴더 내 이미지 파일 목록
        image_files = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(image_files)

        # 훈련/검증 세트 분할
        split_idx = int(len(image_files) * (1 - val_ratio))
        train_files = image_files[:split_idx]
        val_files = image_files[split_idx:]

        # 각 이미지 처리
        for subset, files in [("train", train_files), ("val", val_files)]:
            for img_file in files:
                src_path = os.path.join(class_path, img_file)

                # 이미지 읽기 및 크기 확인
                try:
                    img = Image.open(src_path)
                    width, height = img.size

                    # 파일명 생성 (중복 방지를 위해 카운터 추가)
                    dest_img_name = f"{class_name}_{file_count:05d}.jpg"
                    dest_img_path = os.path.join(output_path, "images", subset, dest_img_name)

                    # 이미지 복사
                    img.save(dest_img_path)

                    # YOLO 형식 레이블 생성 (전체 이미지에 하나의 객체)
                    # 형식: class_id center_x center_y width height (모두 0~1 사이 정규화)
                    class_id = class_mapping[class_name]

                    # 이미지 전체를 바운딩 박스로 설정 (약간의 여백 추가)
                    margin = 0.1
                    x_center, y_center = 0.5, 0.5  # 중앙
                    bbox_width, bbox_height = 1.0 - 2*margin, 1.0 - 2*margin  # 여백 적용

                    # 레이블 파일 생성
                    label_path = os.path.join(output_path, "labels", subset, f"{os.path.splitext(dest_img_name)[0]}.txt")
                    with open(label_path, 'w') as f:
                        f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")

                    file_count += 1
                except Exception as e:
                    print(f"이미지 처리 중 오류: {src_path}, {e}")

# YAML 파일 생성
def create_yaml_file():
    yaml_content = f"""# YOLOv8 dataset config
path: {output_path}
train: images/train
val: images/val

# Classes
names:
  0: rock
  1: paper
  2: scissors
"""

    with open(f"{output_path}/data.yaml", 'w') as f:
        f.write(yaml_content)

# 변환 실행
print("데이터셋 YOLO 형식으로 변환 중...")
convert_to_yolo_format()
create_yaml_file()
print(f"변환 완료. 출력 경로: {output_path}")

# 모델 학습
from ultralytics import YOLO

model1 = YOLO('yolo11n.pt')
results1 = model1.train(
    data=f"{output_path}/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    name='model1_moroney'
)


In [ ]:
# 모델 2: Roboflow의 Rock Paper Scissors 데이터셋

# 코랩에서 실행
!pip install roboflow ultralytics
from roboflow import Roboflow

# Roboflow에서 데이터셋 다운로드
rf = Roboflow(api_key="p2C1Qr5lCgNbiEcup6JK")
project = rf.workspace("joseph-nelson").project("rock-paper-scissors")
dataset = project.version(1).download("yolov8")

# 모델 학습
from ultralytics import YOLO

model2 = YOLO('yolo11n.pt')
results2 = model2.train(
    data=f"{dataset.location}/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    name='model2_roboflow'
)


In [ ]:
# 모델 3: SXSW Rock Paper Scissors 데이터셋

# 코랩에서 실행
# !pip install roboflow ultralytics
# from roboflow import Roboflow

# Roboflow에서 데이터셋 다운로드
rf = Roboflow(api_key="p2C1Qr5lCgNbiEcup6JK")
project = rf.workspace("yolo-2hpdp").project("rock-paper-scissors-sxsw-kljj7")
dataset = project.version(11).download("yolov8")

# 모델 학습
from ultralytics import YOLO

model3 = YOLO('yolo11n.pt')
results3 = model3.train(
    data=f"{dataset.location}/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    name='model3_sxsw'
)


In [ ]:
# 모델 비교 및 평가

# 코랩에서 실행
from ultralytics import YOLO
import matplotlib.pyplot as plt

# 세 모델 로드
model1 = YOLO('runs/detect/model1_moroney/weights/best.pt')
model2 = YOLO('runs/detect/model2_roboflow/weights/best.pt')
model3 = YOLO('runs/detect/model3_sxsw/weights/best.pt')

# 테스트 데이터셋에서 성능 평가
val1 = model1.val()
val2 = model2.val()
val3 = model3.val()

# 결과 비교
models = ['Model 1 (Moroney)', 'Model 2 (Roboflow)', 'Model 3 (SXSW)']
map50 = [val1.box.map50, val2.box.map50, val3.box.map50]
map50_95 = [val1.box.map, val2.box.map, val3.box.map]

plt.figure(figsize=(10, 6))
plt.bar(models, map50, label='mAP@0.5')
plt.bar(models, map50_95, label='mAP@0.5:0.95', alpha=0.7)
plt.ylabel('mAP')
plt.title('Model Comparison')
plt.legend()
plt.savefig('model_comparison.png')
plt.show()

# 최적의 모델 선택 및 저장
best_map = max(map50)
best_model_idx = map50.index(best_map)
best_model_path = [
    'runs/detect/model1_moroney/weights/best.pt',
    'runs/detect/model2_roboflow/weights/best.pt',
    'runs/detect/model3_sxsw/weights/best.pt'
][best_model_idx]

print(f"최적의 모델: {models[best_model_idx]}, mAP@0.5: {best_map}")
print(f"모델 경로: {best_model_path}")

# 최적의 모델을 final_model.pt로 복사
!cp {best_model_path} final_model.pt


In [ ]:
# yolo + skeleton